<a href="https://colab.research.google.com/github/cagBRT/promptEngineering/blob/main/demo_distilgpt2_magicprompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> demo: distilgpt2-magicprompt

- [link to model card](https://huggingface.co/pszemraj/distilgpt2-magicprompt-SD)
- by [Peter](https://github.com/pszemraj)

---

In [1]:
pip install -U -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.5 MB/s eta 0:00:00


In [2]:
import re
from transformers import pipeline

model_id = "pszemraj/distilgpt2-magicprompt-SD"

generator = pipeline(
    "text-generation",
    model=model_id,
    device=-1,  # cpu inference takes 2 seconds
)

In [3]:

#@title define `augment_text` as helper

def clean_text(text):
    text = re.sub(r"\s+", " ", text)  # remove extra whitespace
    text = re.sub(r"\s+([?.!,])", r"\1", text)  # remove whitespace before punctuation
    # remove duplicate punctuation
    text = re.sub(r"([?.!,])\1+", r"\1", text)

    return text

def augment_text(
    prompt: str, 
    pipeline, 
    max_new_tokens: int = 24, 
    clean=True,
    **kwargs,
) -> str:
    """
    augment_text - use a prompt to generate text

    :param str prompt: prompt to augment
    :param pipeline:   pipeline to use for augmentation (text-generation)
    :param int max_new_tokens: maximum number of tokens to generate, default 24
    :param bool clean: clean the generated text, default True
    :return str: augmented text
    """
    result = generator(
        prompt,
        max_new_tokens=max_new_tokens,
        **kwargs,
    )  # generate
    return (
        clean_text(result[0]["generated_text"])
        if clean
        else result[0]["generated_text"]
    )

In [5]:
import time

prompt = "I just ran a mile"  # @param {type:"string"}

kwargs = {
    "clean_up_tokenization_spaces": True,
    # your kwargs/params here (temperature, etc)
}

st = time.perf_counter()
aug_prompt = augment_text(
    prompt=prompt,
    pipeline=generator,
    **kwargs,
)
rt = round(time.perf_counter() - st, 2)

print(f"Runtime: {rt} seconds", "\n" * 2)
aug_prompt

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Runtime: 1.04 seconds 




'I just ran a mile with arms wide open and wearing glasses, sweaty skin without braids,digital art, professional illustration by greg rut'